## Base Goal - Create a balanced protfolio for S&P 500 index
api- iex cloud

## Final goal- For NSE
Data- Scrapping form upstox or google finance

Libraries required->

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module

## Api keys

In [2]:
from keys import key_iex

## External csv

In [3]:
sp500=pd.read_csv('sp_500_stocks.csv')
sp500

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Creating dataframe for conserning values

In [4]:
cols=['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
stock_param=pd.DataFrame(columns=cols )

In [5]:
#This is way of getting calls one by one which is quite slow so we will have to batch requests together for that purpose
# for symbol in sp500['Ticker']:
#     cs=requests.get(f"https://api.iex.cloud/v1/data/core/quote/{symbol}?token={key_iex}").json()[0]
#     stock_param.loc[len(stock_param.index)] = [symbol, 
#                                                    cs['latestPrice'], 
#                                                    cs['marketCap'], 
#                                                    'N/A']


stock_param

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [10]:
from create_batches import create_batches


In [12]:
res=list(create_batches(sp500['Ticker'],100))

In [13]:
batch_req=[]
for i in res:
    batch_req.append(','.join(i))
print(batch_req)

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA', 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD', 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX

In [17]:
#updated way
for symbol in batch_req:
    ot=requests.get(f"https://api.iex.cloud/v1/data/core/quote/{symbol}?token={key_iex}").json()
    for cs in ot:
        stock_param.loc[len(stock_param.index)] = [cs['symbol'], 
                                                    cs['latestPrice'], 
                                                    cs['marketCap'], 
                                                    'N/A']



In [19]:
stock_param.head(10)

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,131.00,38383537362,N/A
1,AAL,13.09,8554845800,N/A
2,AAP,60.49,3599275375,N/A
3,AAPL,181.91,2829201116320,N/A
4,ABBV,161.46,290628000000,N/A
5,ABC,179.98,36387430583,N/A
6,ABMD,381.02,17180642928,N/A
7,ABT,110.98,192667776325,N/A
8,ACN,337.09,214050721075,N/A
9,ADBE,567.05,258177865000,N/A


Now we would like to build a portfolio. So lets start with an amount of 10000000. We will distribute this amount equally among all the stocks to observe the return trends

In [26]:
stock_param['Market Capitalization']=pd.to_numeric(stock_param['Market Capitalization'])
amt=10000000/len(stock_param.index)
for i in range (0,len(stock_param)):
    stock_param.loc[i,"Number Of Shares to Buy"]=math.floor(amt/stock_param.loc[i,"Price"])
stock_param

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,131.00,3.838354e+10,151
1,AAL,13.09,8.554846e+09,1512
2,AAP,60.49,3.599275e+09,327
3,AAPL,181.91,2.829201e+12,108
4,ABBV,161.46,2.906280e+11,122
...,...,...,...,...
500,YUM,128.69,3.607286e+10,153
501,ZBH,120.20,2.511948e+10,164
502,ZBRA,252.97,1.299244e+10,78
503,ZION,42.64,6.317065e+09,464


Now we will need to save the above list to an excel file int formatted data

In [35]:
stock_param.to_csv('stock_prices.csv',header=True,index=False)

In [27]:
writer=pd.ExcelWriter('reccomended_portfolio.xlsx',engine='xlsxwriter')
stock_param.to_excel(writer,sheet_name='portfolio',index=False)

In [28]:
#formats
dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
           
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            
        }
    )

In [30]:
column_formats = { 
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['portfolio'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['portfolio'].write(f'{column}1', column_formats[column][0])


In [31]:
writer.save()

C:\Users\sjmad\AppData\Local\Temp\ipykernel_3416\934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [34]:
print(stock_param.loc[0,'Price'])

131.0
